In [ ]:
# This project detects face mask from images using Computer Vision.
# High-level implementation steps:
# 1. Load and preprocess the dataset
# 2. Build a CNN model using TensorFlow or PyTorch
# 3. Train the model on the dataset
# 4. Implement real-time detection using OpenCV

In [ ]:
# Install kaggle
!pip install kaggle

In [ ]:
# Creating and saving custom mask detection model
# Step 1: Dataset Acquisition

# Mount the Google drive so that you can store your Kaggle API credentials
from google.colab import drive
drive.mount('/content/drive')

# Create a Kaggle directory
!mkdir -p ~/.kaggle

# Copy the kaggle.json file to the Kaggle directory
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/

# Set permissions for the Kaggle API key
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset using the Kaggle API
!kaggle datasets download -d omkargurav/face-mask-dataset

# Unzip the dataset (if needed)
!unzip face-mask-dataset.zip

In [1]:
# Step 2: Preprocess the dataset
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

# Define dataset directories
with_mask_dir = 'data/with_mask'
without_mask_dir = 'data/without_mask'

IMG_WIDTH = 224
IMG_HEIGHT = 224

def load_and_preprocess_images(directory, label):
    images = []
    labels = []
    for img_path in os.listdir(directory):
        try:
            img = cv2.imread(os.path.join(directory, img_path))
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

            # Check if the image was loaded successfully
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue

            img = img_to_array(img)
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue
    return images, labels

# Load images and labels
with_mask_images, with_mask_labels = load_and_preprocess_images(with_mask_dir, 1)
without_mask_images, without_mask_labels = load_and_preprocess_images(without_mask_dir, 0)

# Combine the data
images = with_mask_images + without_mask_images
labels = with_mask_labels + without_mask_labels

# Convert to numpy arrays and normalize
images = np.array(images, dtype="float32") / 255.0
labels = np.array(labels)

print("Number of images:", len(images))
print("Number of labels:", len(labels))
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Number of images: 7553
Number of labels: 7553
Images shape: (7553, 224, 224, 3)
Labels shape: (7553,)


In [ ]:
# Step 3 : Build a CNN Model using Tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

BATCH_SIZE = 32
EPOCHS = 15

# Define data directories
data_dir = 'data/'  # This should contain subdirectories 'with_mask' and 'without_mask'

# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data into training and validation
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=42
)

# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Add EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model using the generators
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping]  # Include EarlyStopping
)

# Save the model
model.save('mask_detection_model.h5')

In [ ]:
# Step 5: Implement detection using OpenCV from images
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load the trained model
model = load_model('mask_detection_model.h5')

# Load the face detection cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_mask(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        face_roi = image[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = img_to_array(face_roi)
        face_roi = preprocess_input(face_roi)
        face_roi = np.expand_dims(face_roi, axis=0)

        prediction = model.predict(face_roi)[0][0]
        label = "Mask" if prediction > 0.5 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

    return image

In [ ]:
# upload the image using google colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# Test on an image
test_image = cv2.imread('<your_image>')
result = detect_mask(test_image)
cv2.imshow('Mask Detection', result)
cv2.waitKey(0)
cv2.destroyAllWindows()